<div style="display:block; margin-bottom:50px">
<h1 style="margin-bottom:25px; font-size:3.5rem;color:#4c76ce;text-align:center;">
    Python Bokeh Data Visualization Tutorial</h1>
    
<h2 style="margin-bottom: 25px;font-size:2.5rem;text-align:center;color:#8d1c1a;">
    Making Iteractions: JavaScript Callbacks</h2>
       
<img src="https://raw.githubusercontent.com/lajmcourses/Images/master/bokeh.png"
     style="position:absolute;top:5px;left:25px;height:150px;width:auto;margin-bottom:35px;">
</div>

While the main goal of Bokeh is to provide a path to create rich interactive visualizations in the browser purely from Python, there will always be specialized use-cases that are outside the capabilities of the core library. For this reason, Bokeh provides different ways for users to supply custom JavaScript when necessary, so that users may add custom or specialized behaviors in response to property changes and other events.

One mechanism is the ability to add entire new custom extension models, as described in Extending Bokeh. However, it is also possible to supply small snippets of JavaScript as callbacks to use, e.g when property values change or when UI or other events occur. This kind of callback can be used to add interesting interactions to Bokeh documents without requiring a Bokeh server (but can also be used in conjunction with a Bokeh server).

In [13]:
from random import random

from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import BoxAnnotation, ColumnDataSource, CustomJS, Slider

from bokeh.plotting import figure


# Display plots in Jupyter Notebook
output_notebook()

Loading BokehJS ...

## CustomJS Callbacks

In [ ]:
callback = CustomJS(args=dict(xr=plot.x_range), code="""

// JavaScript code goes here

const a = 10;

// the model that triggered the callback is cb_obj:
const b = cb_obj.value;

// models passed as args are automagically available
xr.start = a;
xr.end = b;

""")

Note that in addition to the code property, CustomJS also accepts an args property that maps string names to Bokeh models. Any Bokeh models that are configured in args (on the “Python side”) will automatically be available to the JavaScript code by the corresponding name. Additionally, the model that triggers the callback (that is the model that the callback is attached to) will be available as cb_obj.

## CustomJS Callback for a Slider

The first parameter to **js_on_change** is actually the name of a BokehJS event.

In [7]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure, output_file, show


x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(width=400, height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const f = cb_obj.value
    const x = data['x']
    const y = data['y']
    for (let i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)


## CustomJS Callback for Selection

In [10]:
# CustomJS Callback for Selection

## Data
x = [random() for x in range(500)]
y = [random() for y in range(500)]

## Column Data Sources
s1 = ColumnDataSource(data=dict(x=x, y=y))
s2 = ColumnDataSource(data=dict(x=[], y=[]))

## Plot 1
p1 = figure(width=400, height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

## Plot2
p2 = figure(width=400, height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")

p2.circle('x', 'y', source=s2, alpha=0.6)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        const idx = cb_obj.indices;
        const d1 = s1.data;
        const d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (let i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)

layout = row(p1, p2)

show(layout)

## CustomJS for Ranges

The properties of range objects may also be connected to CustomJS callbacks in order to perform specialized work whenever a range changes.

In [14]:
# CustomJS for Ranges

N = 4000

x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = [
    "#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)
]

box = BoxAnnotation(left=0, right=0, bottom=0, top=0,
    fill_alpha=0.1, line_color='black', fill_color='black')

jscode = """
    box[%r] = cb_obj.start
    box[%r] = cb_obj.end
"""

p1 = figure(title='Pan and Zoom Here', x_range=(0, 100), y_range=(0, 100),
            tools='box_zoom,wheel_zoom,pan,reset', width=400, height=400)
p1.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)

xcb = CustomJS(args=dict(box=box), code=jscode % ('left', 'right'))
ycb = CustomJS(args=dict(box=box), code=jscode % ('bottom', 'top'))

p1.x_range.js_on_change('start', xcb)
p1.x_range.js_on_change('end', xcb)
p1.y_range.js_on_change('start', ycb)
p1.y_range.js_on_change('end', ycb)

p2 = figure(title='See Zoom Window Here', x_range=(0, 100), y_range=(0, 100),
            tools='', width=400, height=400)
p2.scatter(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
p2.add_layout(box)

layout = row(p1, p2)

show(layout)